In [ ]:
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import linprog
import SimpleITK as sitk
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import random

np.random.seed(42)

### Loading Data

In [ ]:
# Load the segmentation data
segmentation_map_file_path = "OASIS-TRT-20-10_DKT31_CMA_labels_in_MNI152.nii.gz"
segmentation_map_image = sitk.ReadImage(segmentation_map_file_path)
segmentation_map_data = sitk.GetArrayFromImage(segmentation_map_image) # np.unique(segmentation_map_data) == [0,4,5,6,7,10,11,12,13,14,15,16,...,2030,2031,2034,2035]

# Load the label map of brain
brain_mask_file_path = "Segmentation_8-BrainLabelMap-label_1.nrrd"
brain_mask_image = sitk.ReadImage(brain_mask_file_path)
brain_mask_data = sitk.GetArrayFromImage(brain_mask_image) # np.unique(brain_mask_data) == [0,1]

def assign_random_importance(segmentation_map_data):
    # Get the unique class labels in the segmentation map
    unique_classes = np.unique(segmentation_map_data)
    
    # Remove the background (assuming it is labeled as 0)
    unique_classes = unique_classes[unique_classes != 0]
    
    # Assign a random importance value to each class
    # We can use np.random.rand to generate random values between 0 and 1, or np.random.uniform to specify a range
    importance_map = {}
    for cls in unique_classes:
        importance_map[cls] = np.random.uniform(0, 1)  # Random importance between 0 and 1
    
    return importance_map

# Assign random importance to each class
importance_map = assign_random_importance(segmentation_map_data)

### Utility Functions

In [ ]:
# def add_tumor_to_data(data1, label_map, radius=3):
#     # Find all brain coordinates (label == 1)
#     brain_coords = np.argwhere(label_map == 1)  # Assuming 1 represents brain tissue in your segmentation
    
#     # Randomly select a point as the tumor center
#     tumor_center = brain_coords[np.random.randint(len(brain_coords))]
    
#     # Get the shape of the data matrix
#     shape = data1.shape
    
#     # Add the tumor as a sphere around the tumor center
#     for dz in range(-radius, radius + 1):
#         for dy in range(-radius, radius + 1):
#             for dx in range(-radius, radius + 1):
#                 if dz**2 + dy**2 + dx**2 <= radius**2:  # Check if within the sphere
#                     z, y, x = tumor_center + np.array([dz, dy, dx])
#                     if (
#                         0 <= z < shape[0] and
#                         0 <= y < shape[1] and
#                         0 <= x < shape[2] and
#                         label_map[z, y, x] == 1  # Ensure the tumor stays within brain tissue
#                     ):
#                         data1[z, y, x] = 999  # Assign label 999 for the tumor

#     return data1, tumor_center

# def create_two_sided_beams(tumor_center, num_beams):
#     """
#     Generate two-sided beams originating from the tumor center.
#     Each beam has a direction vector and its opposite.
#     """
#     beams = []

#     # Generate points on a sphere using the Fibonacci sphere method
#     phi = (1 + np.sqrt(5)) / 2  # Golden ratio
#     for i in range(num_beams):
#         z = 1 - (2 * i + 1) / num_beams  # z-coordinates spaced evenly
#         radius = np.sqrt(1 - z**2)  # Radius of the circle at height z
#         theta = 2 * np.pi * i / phi  # Angle using the golden ratio
#         x = radius * np.cos(theta)
#         y = radius * np.sin(theta)

#         direction = np.array([x, y, z])

#         # Each beam has two directions: original and opposite
#         beams.append({"start": tumor_center, "direction": direction})
#         beams.append({"start": tumor_center, "direction": -direction})  # Opposite direction

#     return beams

# def calculate_voxels_affected(beams, data_shape, beam_radius):
#     """
#     Determine which voxels are touched by each beam, given a specific beam radius.
#     """
#     affected_voxels = defaultdict(list)
#     for beam_index, beam in enumerate(beams):
#         start = np.array(beam["start"])
#         direction = np.array(beam["direction"])
        
#         # Move along the beam direction up to the bounds of the 3D space
#         for t in range(-beam_radius, beam_radius + 1):
#             position = start + t * direction
#             voxel = np.round(position).astype(int)  # Convert to voxel indices
            
#             # Ensure voxel is within bounds
#             if all(0 <= voxel[i] < data_shape[i] for i in range(3)):
#                 affected_voxels[beam_index].append(tuple(voxel))
#     return affected_voxels

# def compute_intensity(weights, affected_voxels, data_shape):
#     """
#     Compute the intensity experienced by each voxel based on the beam weights.
#     """
#     intensity_matrix = np.zeros(data_shape)
#     for beam_index, weight in enumerate(weights):
#         for voxel in affected_voxels[beam_index]:
#             z, y, x = voxel

#             intensity = weight  
#             intensity_matrix[z, y, x] += intensity
#     return intensity_matrix

# def calculate_total_tumor_intensity(intensity_matrix, data_with_tumor, tumor_label=999):
#     """
#     Calculate the total intensity delivered to tumor cells.
    
#     Args:
#         intensity_matrix: 3D array of intensity values.
#         data_with_tumor: 3D array with tumor labels (999 for tumor).
#         tumor_label: The label value representing tumor cells (default 999).
    
#     Returns:
#         Total intensity delivered to the tumor cells.
#     """
#     # Create a mask for tumor voxels
#     tumor_mask = (data_with_tumor == tumor_label)
    
#     # Sum the intensities in the tumor region
#     total_tumor_intensity = np.sum(intensity_matrix[tumor_mask])
    
#     return total_tumor_intensity

# # Step 1: Calculate the dose contributions
# def calculate_doses(beams, tumor_mask, normal_mask, intensity_matrix, num_beams, affected_voxels):
#     beam_doses_to_tumor = np.zeros(num_beams)
#     beam_doses_to_normal = np.zeros(num_beams)
#     new_intensity_matrix = np.zeros_like(intensity_matrix)

#     for beam_index, beam in enumerate(beams):
#         for voxel in affected_voxels[beam_index]:
#             z, y, x = voxel

#             intensity = intensity_matrix[z,y,x]
#             new_intensity_matrix[z, y, x] += intensity

#             if tumor_mask[z, y, x]:
#                 beam_doses_to_tumor[beam_index] += intensity
#             elif normal_mask[z, y, x]:
#                 beam_doses_to_normal[beam_index] += intensity

#     return beam_doses_to_tumor, beam_doses_to_normal, new_intensity_matrix

In [ ]:
def add_tumor_to_data(_segmentation_map_data: np.ndarray, _brain_mask_data: np.ndarray, max_tumor_size: int=None):
# Find all brain coordinates (label == 1)
    brain_coords = np.argwhere(_brain_mask_data == 1)  # Assuming 1 represents brain tissue in your segmentation
    
    # Randomly select a point as the tumor center
    tumor_center = brain_coords[np.random.randint(len(brain_coords))]
    
    # Get the shape of the data matrix
    shape = _segmentation_map_data.shape
    
    # Initialize tumor voxels list and add the center voxel
    tumor_voxels = [tuple(tumor_center)]
    _segmentation_map_data[tuple(tumor_center)] = 999  # Assign label 999 for the tumor
    
    # All 8 directions (adjacent + diagonal neighbors)
    directions = [
        (-1, 0, 0), (1, 0, 0),  # x-axis neighbors
        (0, -1, 0), (0, 1, 0),  # y-axis neighbors
        (0, 0, -1), (0, 0, 1),  # z-axis neighbors
        (-1, -1, 0), (1, 1, 0),  # diagonal in xy-plane
        (-1, 0, -1), (1, 0, 1),  # diagonal in xz-plane
        (0, -1, -1), (0, 1, 1),  # diagonal in yz-plane
        (-1, -1, -1), (1, 1, 1), # diagonal in all 3 planes
        (-1, 1, -1), (1, -1, 1), # diagonal in all 3 planes
    ]
    
    # Function to calculate distance from the tumor center
    def distance_from_center(voxel):
        return np.sqrt((voxel[0] - tumor_center[0]) ** 2 + (voxel[1] - tumor_center[1]) ** 2 + (voxel[2] - tumor_center[2]) ** 2)
    
    # Randomly expand the tumor from the seed point
    while len(tumor_voxels) < max_tumor_size if max_tumor_size else True:
        # Randomly select an existing tumor voxel to expand from
        voxel_to_expand = random.choice(tumor_voxels)
        
        # Get the valid neighbors of the voxel
        valid_neighbors = []
        for direction in directions:
            neighbor = tuple(np.array(voxel_to_expand) + np.array(direction))
            z, y, x = neighbor
            
            # Check if the neighbor is within bounds and inside the brain tissue
            if 0 <= z < shape[0] and 0 <= y < shape[1] and 0 <= x < shape[2] and _brain_mask_data[z, y, x] == 1:
                if neighbor not in tumor_voxels:  # Ensure it's not already part of the tumor
                    valid_neighbors.append(neighbor)
        
        # If there are valid neighbors, choose one based on the distance heuristic
        if valid_neighbors:
            # Calculate the distance of each valid neighbor from the center and prefer closer ones
            distances = [distance_from_center(neighbor) for neighbor in valid_neighbors]
            min_distance = min(distances)
            # Select neighbors with distance closest to the tumor center (globular heuristic)
            close_neighbors = [valid_neighbors[i] for i in range(len(valid_neighbors)) if distances[i] == min_distance]
            
            # Randomly pick one of the closest neighbors
            new_voxel = random.choice(close_neighbors)
            tumor_voxels.append(new_voxel)
            _segmentation_map_data[new_voxel] = 999  # Assign label 999 for the tumor
        
        # If no valid neighbors are left, stop growing the tumor
        if not valid_neighbors:
            break
    
    return _segmentation_map_data, tumor_center

def create_single_sided_beams(tumor_center, num_beams, brain_mask_data, data_shape):
    """
    Generate single-sided beams originating from points outside the brain and directed toward the tumor center.
    Each beam has a random direction.
    """
    beams = []
    
    for i in range(num_beams):
        valid_start = False
        while not valid_start:
            # Randomly generate beam starting positions within the data shape
            start = np.array([
                np.random.randint(0, data_shape[0]),  # Random x position
                np.random.randint(0, data_shape[1]),  # Random y position
                np.random.randint(0, data_shape[2]),  # Random z position
            ])

            # Check if the point is outside the brain (brain_mask_data == 0)
            if brain_mask_data[start[0], start[1], start[2]] == 0:
                valid_start = True  # Valid start point outside the brain

        # Direction is towards the tumor center
        direction = tumor_center - start
        direction = direction / np.linalg.norm(direction)  # Normalize direction vector
        
        beams.append({"start": start, "direction": direction})
    
    return beams

def longitudinal_intensity_profile(projected_position, tumor_center, beam_width_longitudinal):
    """
    Customizable intensity profile in the longitudinal direction.
    Uses the Euclidean distance between the projected position and the tumor center.
    """
    distance = np.linalg.norm(projected_position - tumor_center)  # Euclidean distance
    intensity = np.exp(-0.5 * (distance / beam_width_longitudinal) ** 2)  # Gaussian profile
    return intensity

def transverse_intensity_profile(voxel, start, direction, transverse_std_dev):
    """
    Compute the Gaussian intensity in the transverse plane defined by the beam direction.
    The intensity is based on the perpendicular distance from the beam direction.
    """
    # Vector from the beam start to the voxel
    relative_position = voxel - start
    
    # Project relative position onto the beam direction
    projection_length = np.dot(relative_position, direction)
    projected_point = start + projection_length * direction  # Closest point on the beam line

    # Calculate the perpendicular distance to the beam direction
    perpendicular_distance = np.linalg.norm(voxel - projected_point)

    # Gaussian intensity based on the perpendicular distance
    intensity = np.exp(-0.5 * (perpendicular_distance / transverse_std_dev) ** 2)
    return intensity

def compute_intensity_matrix(beams, data_shape, transverse_std_dev, beam_width_longitudinal, tumor_center):
    """
    Compute the intensity matrix for all voxels, considering both transverse and longitudinal intensity profiles.
    """
    intensity_matrix = np.zeros(data_shape)
    
    for beam_index, beam in enumerate(beams):
        start = np.array(beam["start"])
        direction = np.array(beam["direction"])

        # Normalize beam direction to ensure consistent calculations
        direction = direction / np.linalg.norm(direction)

        # Loop over all voxels in the grid
        for z in range(data_shape[0]):  # z-axis loop
            for y in range(data_shape[1]):  # y-axis loop
                for x in range(data_shape[2]):  # x-axis loop
                    voxel = np.array([z, y, x])
                    
                    # Compute transverse and longitudinal intensities
                    intensity_transverse = transverse_intensity_profile(voxel, start, direction, transverse_std_dev)
                    projected_position = start + np.dot(voxel - start, direction) * direction
                    intensity_longitudinal = longitudinal_intensity_profile(projected_position, tumor_center, beam_width_longitudinal)
                    
                    # Combined intensity is the product of both components
                    intensity = intensity_transverse * intensity_longitudinal
                    intensity_matrix[z, y, x] += intensity  # Add intensity to the voxel

    return intensity_matrix

### Problem Definition

In [ ]:
num_beams = 10
beam_radius = 100  # Radius of influence for each beam

# Implant the tumor into the data matrix
data_with_tumor, tumor_center = add_tumor_to_data(segmentation_map_data, brain_mask_data, tumor_radius)

# Generate beams centered at the tumor, with two directions for each beam
beams = create_two_sided_beams(tumor_center, num_beams)

# Calculate which voxels are affected by each beam
affected_voxels = calculate_voxels_affected(beams, data_with_tumor.shape, beam_radius)

# Define intensity weights for each beam (same weight for both directions)
beam_weights = np.ones(len(beams))# Random integers between 1 and 10 (inclusive)

# Compute the intensity matrix
intensity_matrix = compute_intensity(beam_weights, affected_voxels, data_with_tumor.shape)

# Calculate the total intensity going into the tumor cells
total_tumor_intensity = calculate_total_tumor_intensity(intensity_matrix, data_with_tumor)

# Step 2: Initialize masks, weights, and parameters
tumor_mask = (data_with_tumor == 999)
normal_mask = (data_with_tumor != 0) & (data_with_tumor != 999)

weights = np.ones(len(beams))  # Initial weights for the beams
beam_doses_to_tumor, beam_doses_to_normal, new_intensity_matrix = calculate_doses(beams, tumor_mask, normal_mask, intensity_matrix, len(beams), affected_voxels)

### Linear Programming

In [ ]:
# Objective: Minimize the total dose to normal tissue
c = beam_doses_to_normal  # Coefficients for the objective function

# Constraints: Total tumor dose must equal the prescribed dose
A_eq = [beam_doses_to_tumor]
b_eq = [10000]  # Prescribed tumor dose

# Bounds for beam weights (e.g., between 0 and upper_bound)
upper_bound = 50 
bounds = [(0, upper_bound) for _ in range(len(beams))]

# Step 4: Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

# Step 5: Check results
if result.success:
    optimal_beam_weights = result.x
    print("Optimal beam weights:", optimal_beam_weights)
    print("Minimum normal tissue dose:", result.fun)

    calculated_tumor_dose = np.dot(beam_doses_to_tumor, optimal_beam_weights)
    print("Total tumor dose (verification):", calculated_tumor_dose)
else:
    print("Optimization failed:", result.message)

### Evolutionary Algorithm

In [ ]:
# Define objective function
def objective_function(w):
    beam_doses_to_tumor_total = np.dot(beam_doses_to_tumor, w)
    normal_tissue_dose = np.dot(beam_doses_to_normal, w)
    if beam_doses_to_tumor_total < 10000:  # Penalize underdosing
        return normal_tissue_dose + 1000 * (10000 - beam_doses_to_tumor_total)
    return normal_tissue_dose

# Bounds for weights
bounds = [(0, 50) for _ in range(len(beams))]

# Solve with differential evolution
result = differential_evolution(objective_function, bounds)

if result.success:
    print("Optimal weights:", result.x)
    print("Minimum normal tissue dose:", result.fun)
    print("Tumor Tissue Dose:", np.dot(beam_doses_to_tumor, result.x))

else:
    print("Optimization failed:", result.message)

### Pareto Front Multi-objective Optimization

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Define the multi-objective optimization problem
class RadiationOptimization(Problem):
    def __init__(self, beam_doses_to_normal, beam_doses_to_tumor, prescribed_tumor_dose, upper_bound=50):
        super().__init__(n_var=len(beam_doses_to_normal),
                         n_obj=2,
                         n_constr=0,
                         xl=0,  # Lower bounds for beam weights
                         xu=upper_bound)  # Upper bounds for beam weights
        self.beam_doses_to_normal = beam_doses_to_normal
        self.beam_doses_to_tumor = beam_doses_to_tumor
        self.prescribed_tumor_dose = prescribed_tumor_dose

    def _evaluate(self, x, out, *args, **kwargs):
        # Calculate total dose to normal tissue (Objective 1)
        normal_tissue_dose = np.dot(self.beam_doses_to_normal, x.T)

        # Calculate deviation from prescribed tumor dose (Objective 2)
        tumor_dose = np.dot(self.beam_doses_to_tumor, x.T)
        tumor_dose_deviation = (tumor_dose - self.prescribed_tumor_dose)**2

        out["F"] = np.column_stack([normal_tissue_dose, tumor_dose_deviation])

# Problem parameters
prescribed_tumor_dose = 10000
upper_bound = 50

# Create the optimization problem
problem = RadiationOptimization(beam_doses_to_normal, beam_doses_to_tumor, prescribed_tumor_dose, upper_bound)

# Use NSGA-II for multi-objective optimization
algorithm = NSGA2(pop_size=100)

# Perform the optimization
result = minimize(problem, algorithm, termination=('n_gen', 200), seed=1, verbose=True)

# Extract Pareto-optimal solutions
pareto_weights = result.X  # Beam weights
pareto_objectives = result.F  # Corresponding objective values

# Display the Pareto front
print("Pareto-optimal solutions (weights):")
print(pareto_weights)
print("Pareto front (objective values):")
print(pareto_objectives)

# Plot Pareto front
import matplotlib.pyplot as plt
plt.scatter(pareto_objectives[:, 0], pareto_objectives[:, 1], c="blue", label="Pareto front")
plt.xlabel("Total Normal Tissue Dose")
plt.ylabel("Deviation from Target Tumor Dose")
plt.title("Pareto Front for Radiation Beam Optimization")
plt.legend()
plt.grid(True)
plt.show()